In [ ]:
# 02_etl_to_sql.ipynb
# ---
# Marketing Analytics Dashboard - Load Cleaned Data into SQLite and Run Queries

# Step 1: Import libraries
import pandas as pd
import sqlite3
import os

# Step 2: Define file paths
processed_path = '../data/processed/cleaned_marketing_data.csv'
db_path = '../data/marketing_dashboard.db'

# Check if cleaned dataset exists
if not os.path.exists(processed_path):
    raise FileNotFoundError(f"The cleaned dataset was not found at {processed_path}. Please run 01_data_cleaning.ipynb first.")

# Step 3: Load cleaned dataset
df = pd.read_csv(processed_path)
print("Loaded cleaned dataset with shape:", df.shape)

# Step 4: Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
print(f"Connected to database: {db_path}")

# Step 5: Create SQL table and load data
table_name = 'campaign_performance'
df.to_sql(table_name, conn, if_exists='replace', index=False)
print(f"Loaded data into table: {table_name}")

# Step 6: Run sample analytical queries
queries = {
    'Top Campaign Types by ROI': f"""
        SELECT Campaign_Type, ROUND(AVG(ROI), 2) AS Avg_ROI
        FROM {table_name}
        GROUP BY Campaign_Type
        ORDER BY Avg_ROI DESC
        LIMIT 5;
    """,
    'Top Campaigns by Conversion Rate': f"""
        SELECT Company, Campaign_Type, ROUND(AVG(Conversion_Rate)*100, 2) AS Avg_Conversion_Rate
        FROM {table_name}
        GROUP BY Company, Campaign_Type
        ORDER BY Avg_Conversion_Rate DESC
        LIMIT 5;
    """,
    'Overall Channel Performance': f"""
        SELECT Channels_Used, 
               ROUND(AVG(CTR)*100, 2) AS Avg_CTR, 
               ROUND(AVG(CPC), 2) AS Avg_CPC,
               ROUND(AVG(ROI), 2) AS Avg_ROI
        FROM {table_name}
        GROUP BY Channels_Used
        ORDER BY Avg_ROI DESC;
    """,
}

# Step 7: Execute and display results
for name, query in queries.items():
    print(f"\n--- {name} ---")
    result = pd.read_sql_query(query, conn)
    print(result.head())

# Step 8: Close connection
conn.close()
print(f"\nDatabase saved at {db_path} and connection closed.")